### Load packages


In [27]:
import os
import numpy as np
import pandas as pd

## Extract


### Get the raw data


In [28]:
# raw_file_url = 'https://data.calgary.ca/Government/5311-Call-Centre-Raw-dataset/iz86-6y5z/about_data' # if ever needed


raw_file = "/home/joan/Projects/python/call_center_pipeline/data/5311_Call_Centre_-_Raw_dataset_20250901.csv"
if os.path.exists(raw_file):
    df = pd.read_csv(raw_file)
    df.head()
else:
    print(f"File not found: {raw_file}")

## Transform


### Basic data inspection


In [29]:
# Check basic data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     476 non-null    object
 1   Date                   476 non-null    object
 2   APPLICATION            476 non-null    object
 3   OFFERED                476 non-null    int64 
 4   ANSWERED               476 non-null    int64 
 5   AVG ANS DELAY          476 non-null    object
 6   AVG ANS DELAY SECONDS  476 non-null    object
 7   MAX ANS DELAY          476 non-null    object
 8   ABANDONED              476 non-null    int64 
 9   DelayCallsProduct      476 non-null    object
dtypes: int64(3), object(7)
memory usage: 37.3+ KB


In [30]:
# Display head
df.head()

,id,Date,APPLICATION,OFFERED,ANSWERED,AVG ANS DELAY,AVG ANS DELAY SECONDS,MAX ANS DELAY,ABANDONED,DelayCallsProduct
0,2018-02-14-dba_cs_268_5311_gen_inq_script,2018/02/14,dba_cs_268_5311_gen_inq_script,423,375,00:02:44,164,00:09:47,48,"61,500"
1,2018-02-15-dba_cs_268_5311_gen_inq_script,2018/02/15,dba_cs_268_5311_gen_inq_script,395,369,00:00:50,50,00:07:15,26,"18,450"
2,2018-02-16-dba_cs_268_5311_gen_inq_script,2018/02/16,dba_cs_268_5311_gen_inq_script,369,349,00:01:21,81,00:07:14,19,"28,269"
3,2018-02-19-dba_cs_268_5311_gen_inq_script,2018/02/19,dba_cs_268_5311_gen_inq_script,0,0,00:00:00,0,00:00:00,0,0
4,2018-02-20-dba_cs_268_5311_gen_inq_script,2018/02/20,dba_cs_268_5311_gen_inq_script,558,449,00:05:24,324,00:21:48,109,"145,476"


In [31]:
# Summary Statistics
df.describe()

,OFFERED,ANSWERED,ABANDONED
count,476.000000,476.000000,476.000000
mean,294.306723,180.123950,70.115546
std,154.309158,116.931538,51.975108
min,0.000000,0.000000,0.000000
25%,232.750000,99.750000,23.750000
50%,329.000000,167.000000,65.000000
75%,408.000000,284.250000,107.250000
max,672.000000,473.000000,210.000000


### Data Cleaning


In [32]:
# Lets check for missing values
df.isnull().sum()

id                       0
Date                     0
APPLICATION              0
OFFERED                  0
ANSWERED                 0
AVG ANS DELAY            0
AVG ANS DELAY SECONDS    0
MAX ANS DELAY            0
ABANDONED                0
DelayCallsProduct        0
dtype: int64

Lucky us that there are no missing values! Data is clean. Bit of a miracle. We still have lot of work to do.


In [33]:
# Let's check for dupes
duplicates = df[df.duplicated()]


if not duplicates.empty:
    print(f"Dupes found: {len(duplicates)} rows. Please deal with them")
else:
    print("No dupes. We are good to go.")

No dupes. We are good to go.


In [34]:
# Let's transform those columns names to lower case and replace the blank spaces to lower case
print("Original DataFrame columns:")
print(df.columns)

# Convert to lower case
df.columns = df.columns.str.lower()

# Convert to lower case
df.columns = df.columns.str.replace(" ", "_")

print("\nDataFrame columns after converting to lowercase:")
print(df.columns)

Original DataFrame columns:
Index(['id', 'Date', 'APPLICATION', 'OFFERED', 'ANSWERED', 'AVG ANS DELAY',
       'AVG ANS DELAY SECONDS', 'MAX ANS DELAY', 'ABANDONED',
       'DelayCallsProduct'],
      dtype='object')

DataFrame columns after converting to lowercase:
Index(['id', 'date', 'application', 'offered', 'answered', 'avg_ans_delay',
       'avg_ans_delay_seconds', 'max_ans_delay', 'abandoned',
       'delaycallsproduct'],
      dtype='object')


In [35]:
# Let handle that 'delaycallsproduct' column and make
df.rename(columns={"delaycallsproduct": "delay_calls_product"}, inplace=True)
df.columns

Index(['id', 'date', 'application', 'offered', 'answered', 'avg_ans_delay',
       'avg_ans_delay_seconds', 'max_ans_delay', 'abandoned',
       'delay_calls_product'],
      dtype='object')

In [36]:
# Convert the dates into datetime objects
print(df.columns)  # confirm that the column name is 'date'
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Change date format into MM/DD/YYYY
df["date"] = df["date"].dt.strftime("%m/%d/%Y")
print(df["date"].head())

Index(['id', 'date', 'application', 'offered', 'answered', 'avg_ans_delay',
       'avg_ans_delay_seconds', 'max_ans_delay', 'abandoned',
       'delay_calls_product'],
      dtype='object')
0    02/14/2018
1    02/15/2018
2    02/16/2018
3    02/19/2018
4    02/20/2018
Name: date, dtype: object


In [37]:
df.head()

,id,date,application,offered,answered,avg_ans_delay,avg_ans_delay_seconds,max_ans_delay,abandoned,delay_calls_product
0,2018-02-14-dba_cs_268_5311_gen_inq_script,02/14/2018,dba_cs_268_5311_gen_inq_script,423,375,00:02:44,164,00:09:47,48,"61,500"
1,2018-02-15-dba_cs_268_5311_gen_inq_script,02/15/2018,dba_cs_268_5311_gen_inq_script,395,369,00:00:50,50,00:07:15,26,"18,450"
2,2018-02-16-dba_cs_268_5311_gen_inq_script,02/16/2018,dba_cs_268_5311_gen_inq_script,369,349,00:01:21,81,00:07:14,19,"28,269"
3,2018-02-19-dba_cs_268_5311_gen_inq_script,02/19/2018,dba_cs_268_5311_gen_inq_script,0,0,00:00:00,0,00:00:00,0,0
4,2018-02-20-dba_cs_268_5311_gen_inq_script,02/20/2018,dba_cs_268_5311_gen_inq_script,558,449,00:05:24,324,00:21:48,109,"145,476"


In [38]:
# 2) Numeric columns that might come as strings with commas
comma_num_cols = [
    "offered",
    "answered",
    "abandoned",
    "delay_calls_product",
    "avg_ans_delay_seconds",
    # "max_ans_delay",
]
for c in [x for x in comma_num_cols if x in df.columns]:
    df[c] = df[c].astype(str).str.replace(",", "", regex=False).str.strip()
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

In [39]:
# avg_ans_delay -> compute seconds from HH:MM:SS or fall back to any existing numeric column
if "avg_ans_delay" in df.columns:
    # parse hh:mm:ss -> seconds (NaN if parse fails)
    avg_td_secs = pd.to_timedelta(
        df["avg_ans_delay"], errors="coerce"
    ).dt.total_seconds()
    # try to use an existing numeric column if provided
    if "avg_ans_delay_seconds" in df.columns:
        avg_num = pd.to_numeric(
            df["avg_ans_delay_seconds"].astype(str).str.replace(",", ""),
            errors="coerce",
        )
        df["avg_ans_delay_seconds"] = (
            avg_td_secs.fillna(avg_num).fillna(0).round().astype(int)
        )
    else:
        df["avg_ans_delay_seconds"] = avg_td_secs.fillna(0).round().astype(int)

# max_ans_delay -> try parsing HH:MM:SS first, otherwise use numeric (strip commas)
if "max_ans_delay" in df.columns:
    max_td_secs = pd.to_timedelta(
        df["max_ans_delay"], errors="coerce"
    ).dt.total_seconds()
    max_num = pd.to_numeric(
        df["max_ans_delay"].astype(str).str.replace(",", ""), errors="coerce"
    )
    df["max_ans_delay"] = max_td_secs.fillna(max_num).fillna(0).round().astype(int)

In [40]:
# Quick sanity check
print(
    df[
        [
            "avg_ans_delay",
            "avg_ans_delay_seconds",
            "max_ans_delay",
            "delay_calls_product",
        ]
    ].head()
)

  avg_ans_delay  avg_ans_delay_seconds  max_ans_delay  delay_calls_product
0      00:02:44                    164            587                61500
1      00:00:50                     50            435                18450
2      00:01:21                     81            434                28269
3      00:00:00                      0              0                    0
4      00:05:24                    324           1308               145476


In [41]:
# Drop unnecessary columns
df = df.drop(columns=["avg_ans_delay"], errors="ignore")

# Rename 'max_ans_delay" for consistency and delay_calls_product to "weighted_wait_time" for a clearer picture of how our call center is going
df.rename(
    columns={
        "max_ans_delay": "max_wait_time_seconds",
        "delay_calls_product": "weighted_wait_time",

    },
    inplace=True,
)
df.columns


df.head()

,id,date,application,offered,answered,avg_ans_delay_seconds,max_wait_time_seconds,abandoned,weighted_wait_time
0,2018-02-14-dba_cs_268_5311_gen_inq_script,02/14/2018,dba_cs_268_5311_gen_inq_script,423,375,164,587,48,61500
1,2018-02-15-dba_cs_268_5311_gen_inq_script,02/15/2018,dba_cs_268_5311_gen_inq_script,395,369,50,435,26,18450
2,2018-02-16-dba_cs_268_5311_gen_inq_script,02/16/2018,dba_cs_268_5311_gen_inq_script,369,349,81,434,19,28269
3,2018-02-19-dba_cs_268_5311_gen_inq_script,02/19/2018,dba_cs_268_5311_gen_inq_script,0,0,0,0,0,0
4,2018-02-20-dba_cs_268_5311_gen_inq_script,02/20/2018,dba_cs_268_5311_gen_inq_script,558,449,324,1308,109,145476


In [42]:
"""
Let's add some enhancements to the data.

We will add the following KPIs:

- SLA: Industry average of 20 seconds
- Calls answered within SLA
- Call answer rate
- Call abandonment rate
- Completion rate
- Average wait per offered call
- Abandon-to-answer ratio
"""

# SLA Threshold (in seconds)
SLA_THRESHOLD = 20

# Answer rate (answered / offered)
df["answer_rate"] = (
    (df["answered"] / df["offered"])
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round(2)
)

# Abandonment rate (abandoned / offered)
df["abandonment_rate"] = (
    (df["abandoned"] / df["offered"])
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round(2)
)

# Completion rate (answered / (answered + abandoned))
df["completion_rate"] = (
    (df["answered"] / (df["answered"] + df["abandoned"]))
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round(2)
)

# Average wait per offered call (weighted_wait_time / offered)
df["avg_wait_per_call_seconds"] = (
    (df["weighted_wait_time"] / df["offered"])
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round()
    .astype(int)
)

# Abandon-to-answer ratio (abandoned / answered)
df["aband_to_answer_ratio"] = (
    (df["abandoned"] / df["answered"])
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round(2)
)

# Above or bellow sites averager wait per call
site_avg_wait_per_call = round(df["avg_wait_per_call_seconds"].mean())
print(site_avg_wait_per_call)

df["above_site_avg_wait_per_call_seconds"] = (
    df["max_wait_time_seconds"] > site_avg_wait_per_call
)

# --- SLA KPIs ---

# Calls answered within SLA (count)
df["answered_within_sla"] = df.apply(
    lambda row: (
        row["answered"] if row["avg_wait_per_call_seconds"] <= SLA_THRESHOLD else 0
    ),
    axis=1,
)

# SLA compliance (answered within SLA / answered)
df["sla_compliance_rate"] = (
    (df["answered_within_sla"] / df["answered"])
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round(2)
)

# SLA compliance vs offered (answered within SLA / offered)
df["sla_compliance_vs_offered"] = (
    (df["answered_within_sla"] / df["offered"])
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round(2)
)


df.sample(6)

148


,id,date,application,offered,answered,avg_ans_delay_seconds,max_wait_time_seconds,abandoned,weighted_wait_time,answer_rate,abandonment_rate,completion_rate,avg_wait_per_call_seconds,aband_to_answer_ratio,above_site_avg_wait_per_call_seconds,answered_within_sla,sla_compliance_rate,sla_compliance_vs_offered
16,2018-03-09-dba_cs_268_5311_gen_inq_script,03/09/2018,dba_cs_268_5311_gen_inq_script,379,290,183,1015,62,53070,0.77,0.16,0.82,140,0.21,True,0,0.0,0.0
45,2018-04-19-dba_cs_268_5311_gen_inq_script,04/19/2018,dba_cs_268_5311_gen_inq_script,414,316,242,863,58,76472,0.76,0.14,0.84,185,0.18,True,0,0.0,0.0
77,2018-06-04-dba_cs_268_5311_gen_inq_script,06/04/2018,dba_cs_268_5311_gen_inq_script,520,427,330,1124,85,140910,0.82,0.16,0.83,271,0.20,True,0,0.0,0.0
218,2018-12-18-dba_cs_268_5311_gen_inq_script,12/18/2018,dba_cs_268_5311_gen_inq_script,274,236,80,604,18,18880,0.86,0.07,0.93,69,0.08,True,0,0.0,0.0
229,2019-01-02-dba_cs_268_5311_gen_inq_script,01/02/2019,dba_cs_268_5311_gen_inq_script,272,209,87,771,32,18183,0.77,0.12,0.87,67,0.15,True,0,0.0,0.0
221,2018-12-21-dba_cs_268_5311_gen_inq_script,12/21/2018,dba_cs_268_5311_gen_inq_script,192,132,129,1081,25,17028,0.69,0.13,0.84,89,0.19,True,0,0.0,0.0


Now let's add some anomaly detection. We will do it manually for now. We will use the IQR method to detect outliers in the `value` column. Later on we will use a more sophisticated method.

In [43]:
# Rolling baseline for call volume anomalies
df["call_volume_mean"] = round(df["offered"].rolling(7).mean(), 2)
df["call_volume_std"] = round(df["offered"].rolling(7).std(), 2)
df["call_volume_spike"] = df["offered"] > (df["call_volume_mean"] + 2 * df["call_volume_std"])

# Excessive wait anomalies
threshold_wait = df["avg_wait_per_call_seconds"].quantile(0.95)
df["excessive_wait"] = df["avg_wait_per_call_seconds"] > threshold_wait

# SLA compliance anomalies
df["sla_breach"] = df["sla_compliance_rate"] < 0.80

df.sample(10)

,id,date,application,offered,answered,avg_ans_delay_seconds,max_wait_time_seconds,abandoned,weighted_wait_time,answer_rate,...,aband_to_answer_ratio,above_site_avg_wait_per_call_seconds,answered_within_sla,sla_compliance_rate,sla_compliance_vs_offered,call_volume_mean,call_volume_std,call_volume_spike,excessive_wait,sla_breach
179,2018-10-24-dba_cs_268_5311_gen_inq_script,10/24/2018,dba_cs_268_5311_gen_inq_script,361,105,403,2355,114,42315,0.29,...,1.09,True,0,0.0,0.0,399.14,44.76,False,False,True
347,2019-06-18-dba_cs_268_5311_gen_inq_script,06/18/2019,dba_cs_268_5311_gen_inq_script,324,117,436,1824,123,51012,0.36,...,1.05,True,0,0.0,0.0,362.43,44.91,False,False,True
396,2019-08-26-dba_cs_268_5311_gen_inq_script,08/26/2019,dba_cs_268_5311_gen_inq_script,134,108,133,747,16,14364,0.81,...,0.15,True,0,0.0,0.0,138.14,18.86,False,False,True
157,2018-09-24-dba_cs_268_5311_gen_inq_script,09/24/2018,dba_cs_268_5311_gen_inq_script,442,371,142,1001,47,52682,0.84,...,0.13,True,0,0.0,0.0,391.14,36.76,False,False,True
230,2019-01-03-dba_cs_268_5311_gen_inq_script,01/03/2019,dba_cs_268_5311_gen_inq_script,282,98,264,2711,90,25872,0.35,...,0.92,True,0,0.0,0.0,146.71,114.69,False,False,True
213,2018-12-11-dba_cs_268_5311_gen_inq_script,12/11/2018,dba_cs_268_5311_gen_inq_script,324,187,129,763,61,24123,0.58,...,0.33,True,0,0.0,0.0,310.00,56.06,False,False,True
303,2019-04-17-dba_cs_268_5311_gen_inq_script,04/17/2019,dba_cs_268_5311_gen_inq_script,387,254,123,1708,72,31242,0.66,...,0.28,True,0,0.0,0.0,358.14,33.85,False,False,True
242,2019-01-21-dba_cs_268_5311_gen_inq_script,01/21/2019,dba_cs_268_5311_gen_inq_script,355,207,268,1236,89,55476,0.58,...,0.43,True,0,0.0,0.0,327.14,37.41,False,False,True
286,2019-03-22-dba_cs_268_5311_gen_inq_script,03/22/2019,dba_cs_268_5311_gen_inq_script,300,108,347,2172,106,37476,0.36,...,0.98,True,0,0.0,0.0,342.57,36.52,False,False,True
171,2018-10-12-dba_cs_268_5311_gen_inq_script,10/12/2018,dba_cs_268_5311_gen_inq_script,363,120,470,1560,109,56400,0.33,...,0.91,True,0,0.0,0.0,325.00,149.06,False,False,True
